# Investigation of ODI Cricket Data Through Machine Learning

With the 2019 ICC Cricket World Cup now in full swing, I wanted to dig a little deeper into the statistics behind this weird and wonderful game.

There is a vast amount of data collected within cricket. Every ball bowled represents a potentially interesting data point (runs scored, wickets taken), and each has the potential to follow patterns. This project will attempt to identify these patterns through various machine learning techniques, and see what it really takes to win a cricket match.

In the One Day Internation (ODI) format, the team batting first will have 50 overs (6 balls in an over) to amas as higher score as possible whilst the other team attempt to take wickets and limit their score. The second team will then have 50 overs to try and chase down that score. Either team can lose a maximum of 10 wickets within the 50 over innings.

## Data



The dataset we are focussing on is available to download from [this](https://www.kaggle.com/venky73/icc-cricket-world-cup-2019-analysis) Kaggle kernel. The data has been scraped from [https://www.espncricinfo.com/](https://www.espncricinfo.com/) and split into 6 dataframes contained within .csv files. File overview:
1. **ODI_Match_Results.csv** - Contains data regarding the teams involved, who won the toss and whether they chose to bat first or second, what the outcome of the match was, where and when it was played.
1. **ODI_Match_Totals.csv** - Contains data regarding the scores achieved by each team, winning margin, match result, and some data regarding the teams involved, when and where the match played is repeated here.
1. **WC_players.csv** - Contains player information.
1. **Ground_Averages.csv** - Contains aggregated data from various cricket grounds around the world.
1. **Bowler_data.csv** - Contains bowling figures such as balls bowled, wickets taken and bowling economoy.
1. **Batsman_Data.csv** - Contains batting data such as balls faced, runs scored, strike rate, number of boundaries scored.

Initial analysis will focus on the first two dataframes to build up an insight on the following questions:
1. Which team has the highest win percentage?
1. Which teams are most successful in England?
1. Given that a team wins the toss, what is the probability that they win the game?
1. What team is most successful if a toss is won, and which is most successful if a toss is lost?
1. What is the most common choice for each of the English grounds?
1. What is the probability that a team wins their next game, given that the result of their previous game?

Linear Regression models:
* Win predictor
    * Using game stats
    * Using batsmen/ bowler stats
    * Using all stats
* Score predictor

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

df_match = pd.read_csv('../Data/raw/ODI_Match_Results.csv')
df_totals = pd.read_csv('../Data/raw/ODI_Match_Totals.csv')
df_players = pd.read_csv('../Data/raw/WC_players.csv')
df_ground = pd.read_csv('../Data/raw/Ground_Averages.csv')
df_bowl = pd.read_csv('../Data/raw/Bowler_data.csv')
df_bat = pd.read_csv('../Data/raw/Batsman_Data.csv')

### Assess Data Cleanliness & Tidiness

In [2]:
df_match.shape, df_totals.shape, df_players.shape, df_ground.shape, df_bowl.shape, df_bat.shape

((1322, 12), (1296, 13), (152, 3), (106, 11), (11118, 14), (11149, 13))

> We've read all the available dataframes and looked at their shapes.
>
> The 2 main datasets we're interested in for this analysis are the df_match and df_result. Lets have a quick look at these below.

In [3]:
df_match

,Unnamed: 0,Result,Margin,BR,Toss,Bat,Opposition,Ground,Start Date,Match_ID,Country,Country_ID
0,418,won,85 runs,NaN,lost,1st,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,692,lost,85 runs,NaN,won,2nd,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,419,lost,10 runs,NaN,lost,2nd,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,693,won,10 runs,NaN,won,1st,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,121,lost,107 runs,NaN,lost,2nd,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8
5,1096,won,107 runs,NaN,won,1st,v Sri Lanka,Melbourne,11 Jan 2013,ODI # 3317,Australia,2
6,694,lost,9 runs,NaN,lost,2nd,v England,Rajkot,11 Jan 2013,ODI # 3318,India,6
7,852,won,9 runs,NaN,won,1st,v India,Rajkot,11 Jan 2013,ODI # 3318,England,1
8,122,won,8 wickets,59.0,won,2nd,v Australia,Adelaide,13 Jan 2013,ODI # 3319,SriLanka,8
9,1097,lost,8 wickets,59.0,lost,1st,v Sri Lanka,Adelaide,13 Jan 2013,ODI # 3319,Australia,2


In [293]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322 entries, 0 to 1321
Data columns (total 12 columns):
Unnamed: 0    1322 non-null int64
Result        1322 non-null object
Margin        1322 non-null object
BR            606 non-null float64
Toss          1322 non-null object
Bat           1322 non-null object
Opposition    1322 non-null object
Ground        1322 non-null object
Start Date    1322 non-null object
Match_ID      1322 non-null object
Country       1322 non-null object
Country_ID    1322 non-null int64
dtypes: float64(1), int64(2), object(9)
memory usage: 124.0+ KB


In [294]:
df_match.dtypes

Unnamed: 0      int64
Result         object
Margin         object
BR            float64
Toss           object
Bat            object
Opposition     object
Ground         object
Start Date     object
Match_ID       object
Country        object
Country_ID      int64
dtype: object

In [295]:
df_match.Match_ID.value_counts()
df_match.shape[0] - df_match.Match_ID.duplicated(keep=False).sum(), df_match.Match_ID.duplicated().sum()

(196, 563)

In [4]:
df_match.Match_ID.duplicated(keep=False).sum()

1126

- BR is a non-descriptive heading for balls remaining, some columns headings have white spaces.
- BR has a lot of null values. All other columns have no null values.
- Winning margin is given by runs and by wickets, and the data type is string. We'll need to separate these statsictics out and convert to integer.
- 563 matches are represented by 2 rows of data, and 196 matches are represented by a single row of data. It would be tidier if we represented each game in a single row of data.
- Start date is not in datetime format
- Match_ID has a standard format starting with 'ODI #', we can remove this prefix.
- Opposition country in format "v Country", we can remove the "v " prefix
- Unnamed: 0 likely was an index value and can be dropped.
- There are multiple results likely to mean match abandoned. \['n/r', 'aban', 'canc'\]. 
- 97 matches have winning margin as '-', lets replace with NaN
- There are more instances of '-' values in other columns in result,  toss and bat columns too.
- Binary series 'toss' and 'bat' are type `object`, we'll make these represent booleans by 1,0 assignment

### Clean & Test

#### Define
- Make all columns lower case with whitespace removed
- Change values of '-' to NaN
- Make result values \['n/r', 'aban', 'canc', '-'] common, n/r
- Split margin collumn into win_margin and win_method
- Convert win_margin from string to float. We're using float here to avoid errors when converting NaN.
- Change start date to datetime format
- Remove 'v ' prefix from opposition series
- Remove 'ODI # ' prefix from match_id series
- Drop unnamed: 0 column

In [296]:
def renameCols(df):
    '''
    Make all column headers in df lower case with whitespace removed.
    Returns None.
    '''
    df.rename(lambda c: c.lower().replace(' ', '_'), axis=1, inplace=True)

renameCols(df_match)
df_match.head()

,unnamed:_0,result,margin,br,toss,bat,opposition,ground,start_date,match_id,country,country_id
0,418,won,85 runs,NaN,lost,1st,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,692,lost,85 runs,NaN,won,2nd,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,419,lost,10 runs,NaN,lost,2nd,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,693,won,10 runs,NaN,won,1st,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,121,lost,107 runs,NaN,lost,2nd,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


> Column names have been changed successfully.

In [297]:
df_match.dtypes

unnamed:_0      int64
result         object
margin         object
br            float64
toss           object
bat            object
opposition     object
ground         object
start_date     object
match_id       object
country        object
country_id      int64
dtype: object

In [298]:
def countVal (df, s):
    ''' (df, str) -> list of dct
    Counts instances of s for each column in df with type object. Returns counts for each column
    as a list of dct.
    '''
    return [{c:(df[c].str.count('(^)'+s+'($)')).sum()} for c in df.loc[:, df.dtypes==object].columns]

countVal(df_match, '-')

[{'result': 3},
 {'margin': 97},
 {'toss': 26},
 {'bat': 30},
 {'opposition': 0},
 {'ground': 0},
 {'start_date': 0},
 {'match_id': 0},
 {'country': 0}]

In [299]:
def replaceNaN (df, col_list, str):
    ''' (df, list of str, str) -> df
    Replace values equal to str in df['col'] with NaN.
    '''
    for col in col_list:
        df[col] = np.where(df[col] == str, np.NaN, df[col])

col_list  = df_match.loc[:, df_match.dtypes==object].columns
replaceNaN(df_match, col_list, '-')

for col in col_list:
    assert (df_match[col] != '-').all()

countVal(df_match, '-')

[{'result': 0.0},
 {'margin': 0.0},
 {'toss': 0.0},
 {'bat': 0.0},
 {'opposition': 0},
 {'ground': 0},
 {'start_date': 0},
 {'match_id': 0},
 {'country': 0}]

> We do not get an assertion error when testing for '-' values in all columns of type `object` and `countVal` returns zero for all of these colums.
>
> Checking other columns for '-' raises a TypeError, however we know that if there were any '-' values present the entire series would be of type `object` so we know this is not the case.
>
> We have introduced more NaN values so lets take another look at the structure of our data.

In [300]:
df_match.info()
df_match.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322 entries, 0 to 1321
Data columns (total 12 columns):
unnamed:_0    1322 non-null int64
result        1319 non-null object
margin        1225 non-null object
br            606 non-null float64
toss          1296 non-null object
bat           1292 non-null object
opposition    1322 non-null object
ground        1322 non-null object
start_date    1322 non-null object
match_id      1322 non-null object
country       1322 non-null object
country_id    1322 non-null int64
dtypes: float64(1), int64(2), object(9)
memory usage: 124.0+ KB


unnamed:_0      0
result          3
margin         97
br            716
toss           26
bat            30
opposition      0
ground          0
start_date      0
match_id        0
country         0
country_id      0
dtype: int64

In [301]:
no_results = ['n/r', 'aban', 'canc']

def noResult(df, col, lst, new_val):
    '''
    For df['col'], make all values in lst equal to new_val.
    '''
    for val in lst:
        df[col].replace(lst, new_val, inplace=True)
        
noResult(df_match, 'result', no_results, 'n/r')

for val in no_results[1:]:
    assert (df_match.result != val).all() 
    
df_match.result.value_counts()

won     663
lost    562
n/r      80
tied     14
Name: result, dtype: int64

> n/r values now common, there are 80 in total.

In [302]:
def splitCol(df, col, new_col1, new_col2):
    '''
    Extracts numerical digits in df[col] into new_col1 then converts them to 
    type float. Extarcts alphabetical chars from df[col] into new_col2.
    Drops col from df.
    '''
    df[new_col1] = df.margin.str.extract('(\d+)').astype(float)
    df[new_col2] = df.margin.str.extract('([a-zA-Z]+)')
    df.drop(col, axis=1, inplace=True)

splitCol(df_match, 'margin', 'win_margin', 'win_method')
df_match.head()

,unnamed:_0,result,br,toss,bat,opposition,ground,start_date,match_id,country,country_id,win_margin,win_method
0,418,won,NaN,lost,1st,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7,85.0,runs
1,692,lost,NaN,won,2nd,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6,85.0,runs
2,419,lost,NaN,lost,2nd,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7,10.0,runs
3,693,won,NaN,won,1st,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6,10.0,runs
4,121,lost,NaN,lost,2nd,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8,107.0,runs


In [303]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322 entries, 0 to 1321
Data columns (total 13 columns):
unnamed:_0    1322 non-null int64
result        1319 non-null object
br            606 non-null float64
toss          1296 non-null object
bat           1292 non-null object
opposition    1322 non-null object
ground        1322 non-null object
start_date    1322 non-null object
match_id      1322 non-null object
country       1322 non-null object
country_id    1322 non-null int64
win_margin    1225 non-null float64
win_method    1225 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 134.3+ KB


> `splitCol` has functioned correctly, separating the winning margain and winning method (by runs or by wickets) and converting the win_margain into type float. Notice we still have 97 NaN values in each series as expected.

In [168]:
df_match.start_date = pd.to_datetime(df_match.start_date)
df_match.head()

,unnamed:_0,result,br,toss,bat,opposition,ground,start_date,match_id,country,country_id,win_margin,win_method
0,418,won,NaN,lost,1st,v India,Kolkata,2013-01-03,ODI # 3315,Pakistan,7,85.0,runs
1,692,lost,NaN,won,2nd,v Pakistan,Kolkata,2013-01-03,ODI # 3315,India,6,85.0,runs
2,419,lost,NaN,lost,2nd,v India,Delhi,2013-01-06,ODI # 3316,Pakistan,7,10.0,runs
3,693,won,NaN,won,1st,v Pakistan,Delhi,2013-01-06,ODI # 3316,India,6,10.0,runs
4,121,lost,NaN,lost,2nd,v Australia,Melbourne,2013-01-11,ODI # 3317,SriLanka,8,107.0,runs


In [169]:
def matchPrefix(df, col):
    '''
    Extract only digits from each value in df[col] as int and reassign to df[col]
    '''
    df[col] = df[col].str.extract('(\d+)').astype(int)
    
matchPrefix(df_match, 'match_id')
df_match.head()

,unnamed:_0,result,br,toss,bat,opposition,ground,start_date,match_id,country,country_id,win_margin,win_method
0,418,won,NaN,lost,1st,v India,Kolkata,2013-01-03,3315,Pakistan,7,85.0,runs
1,692,lost,NaN,won,2nd,v Pakistan,Kolkata,2013-01-03,3315,India,6,85.0,runs
2,419,lost,NaN,lost,2nd,v India,Delhi,2013-01-06,3316,Pakistan,7,10.0,runs
3,693,won,NaN,won,1st,v Pakistan,Delhi,2013-01-06,3316,India,6,10.0,runs
4,121,lost,NaN,lost,2nd,v Australia,Melbourne,2013-01-11,3317,SriLanka,8,107.0,runs


In [170]:
def oppPrefix(df, col):
    '''
    Remove first 3 chars from each value in df[col]
    '''
    df[col] = df[col].str[2:]
    
oppPrefix(df_match, 'opposition')
df_match.head()

,unnamed:_0,result,br,toss,bat,opposition,ground,start_date,match_id,country,country_id,win_margin,win_method
0,418,won,NaN,lost,1st,India,Kolkata,2013-01-03,3315,Pakistan,7,85.0,runs
1,692,lost,NaN,won,2nd,Pakistan,Kolkata,2013-01-03,3315,India,6,85.0,runs
2,419,lost,NaN,lost,2nd,India,Delhi,2013-01-06,3316,Pakistan,7,10.0,runs
3,693,won,NaN,won,1st,Pakistan,Delhi,2013-01-06,3316,India,6,10.0,runs
4,121,lost,NaN,lost,2nd,Australia,Melbourne,2013-01-11,3317,SriLanka,8,107.0,runs


In [171]:
df_match.drop('unnamed:_0', inplace=True, axis=1)
df_match.head()

,result,br,toss,bat,opposition,ground,start_date,match_id,country,country_id,win_margin,win_method
0,won,NaN,lost,1st,India,Kolkata,2013-01-03,3315,Pakistan,7,85.0,runs
1,lost,NaN,won,2nd,Pakistan,Kolkata,2013-01-03,3315,India,6,85.0,runs
2,lost,NaN,lost,2nd,India,Delhi,2013-01-06,3316,Pakistan,7,10.0,runs
3,won,NaN,won,1st,Pakistan,Delhi,2013-01-06,3316,India,6,10.0,runs
4,lost,NaN,lost,2nd,Australia,Melbourne,2013-01-11,3317,SriLanka,8,107.0,runs


In [172]:
df_match.duplicated().sum()
df_match.bat.value_counts()

1st    656
2nd    636
Name: bat, dtype: int64

In [173]:
df_match['team_won'] = np.where(df_match.result == 'won', df_match.country, df_match.opposition)
df_match['team_won'] = np.where(df_match.result == 'tied', 'tied', df_match.team_won)
df_match['team_won'] = np.where(df_match.result == 'n/r', 'n/r', df_match.team_won)
df_match['won_toss'] = np.where(df_match.toss == 'won', 1, 0)
df_match['bat_1st'] = np.where(df_match.bat == '1st', 1, 0)
df_match.head()

,result,br,toss,bat,opposition,ground,start_date,match_id,country,country_id,win_margin,win_method,team_won,won_toss,bat_1st
0,won,NaN,lost,1st,India,Kolkata,2013-01-03,3315,Pakistan,7,85.0,runs,Pakistan,0,1
1,lost,NaN,won,2nd,Pakistan,Kolkata,2013-01-03,3315,India,6,85.0,runs,Pakistan,1,0
2,lost,NaN,lost,2nd,India,Delhi,2013-01-06,3316,Pakistan,7,10.0,runs,India,0,0
3,won,NaN,won,1st,Pakistan,Delhi,2013-01-06,3316,India,6,10.0,runs,India,1,1
4,lost,NaN,lost,2nd,Australia,Melbourne,2013-01-11,3317,SriLanka,8,107.0,runs,Australia,0,0


In [174]:
df_match.rename(columns={'br':'balls_remaining','opposition':'country2','country':'country1'}, inplace=True)

In [175]:
cols = ['match_id', 'country1', 'country2', 'team_won', 'toss', 'bat', 'win_margin', 'win_method', 'balls_remaining',
 'ground', 'start_date', 'country_id', 'result']
df_match = df_match[cols]
df_match.head()

,match_id,country1,country2,team_won,toss,bat,win_margin,win_method,balls_remaining,ground,start_date,country_id,result
0,3315,Pakistan,India,Pakistan,lost,1st,85.0,runs,NaN,Kolkata,2013-01-03,7,won
1,3315,India,Pakistan,Pakistan,won,2nd,85.0,runs,NaN,Kolkata,2013-01-03,6,lost
2,3316,Pakistan,India,India,lost,2nd,10.0,runs,NaN,Delhi,2013-01-06,7,lost
3,3316,India,Pakistan,India,won,1st,10.0,runs,NaN,Delhi,2013-01-06,6,won
4,3317,SriLanka,Australia,Australia,lost,2nd,107.0,runs,NaN,Melbourne,2013-01-11,8,lost


In [176]:
duplicates = df_match[df_match.match_id.duplicated()].index
df_match.drop(duplicates, axis=0, inplace=True)

In [177]:
df_match.shape

(743, 13)

In [178]:
def dupIndex (df, col):
    ''' (df, str)->list of int
    Returns index values for all duplicate values in df[col] not including
    the 1st instance.
    '''
    return np.array(df[df[col].duplicated()].index)

dupIndex(df_match, 'match_id')

array([], dtype=int64)

In [179]:
np.array(df_match[df_match['match_id'].duplicated()].index)

array([], dtype=int64)

In [180]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 0 to 1320
Data columns (total 13 columns):
match_id           743 non-null int64
country1           743 non-null object
country2           743 non-null object
team_won           743 non-null object
toss               742 non-null object
bat                740 non-null object
win_margin         702 non-null float64
win_method         702 non-null object
balls_remaining    348 non-null float64
ground             743 non-null object
start_date         743 non-null datetime64[ns]
country_id         743 non-null int64
result             741 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(8)
memory usage: 81.3+ KB


In [181]:
df_match.isna().sum()

match_id             0
country1             0
country2             0
team_won             0
toss                 1
bat                  3
win_margin          41
win_method          41
balls_remaining    395
ground               0
start_date           0
country_id           0
result               2
dtype: int64

In [182]:
df_match.to_csv('../Data/interim/ODI_match_clean.csv', index=False)

## Assess Data Cleanliness & Tidiness

Now that we have cleaned df_match and exported as a clean.csv file we'll assess the cleanliness of the df_totals and implement the cleaning scripts used previously.

In [5]:
df_totals

,Unnamed: 0,Score,Overs,RPO,Target,Inns,Result,Opposition,Ground,Start Date,Match_ID,Country,Country_ID
0,412,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,680,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,413,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,681,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,117,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8
5,1076,305/5,50.0,6.10,NaN,1,won,v Sri Lanka,Melbourne,11 Jan 2013,ODI # 3317,Australia,2
6,682,316/9,50.0,6.32,326.0,2,lost,v England,Rajkot,11 Jan 2013,ODI # 3318,India,6
7,836,325/4,50.0,6.50,NaN,1,won,v India,Rajkot,11 Jan 2013,ODI # 3318,England,1
8,118,172/2,40.1,4.28,171.0,2,won,v Australia,Adelaide,13 Jan 2013,ODI # 3319,SriLanka,8
9,1077,170,46.5,3.62,NaN,1,lost,v Sri Lanka,Adelaide,13 Jan 2013,ODI # 3319,Australia,2


In [184]:
df_totals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 13 columns):
Unnamed: 0    1296 non-null int64
Score         1296 non-null object
Overs         1296 non-null float64
RPO           1296 non-null object
Target        620 non-null float64
Inns          1296 non-null int64
Result        1296 non-null object
Opposition    1296 non-null object
Ground        1296 non-null object
Start Date    1296 non-null object
Match_ID      1296 non-null object
Country       1296 non-null object
Country_ID    1296 non-null int64
dtypes: float64(2), int64(3), object(8)
memory usage: 131.7+ KB


In [185]:
df_totals.Inns.value_counts()

1    656
2    620
0     20
Name: Inns, dtype: int64

In [186]:
df_totals.Score.values

array(['250', '165', '157', ..., '120/8', '193/6', '351/9'], dtype=object)

In [187]:
df_totals.Match_ID.duplicated().sum(), df_totals.shape[0] - df_totals.Match_ID.duplicated(keep=False).sum()

(554, 188)

In [188]:
countVal(df_totals,'-')

[{'Score': 0},
 {'RPO': 20},
 {'Result': 3},
 {'Opposition': 0},
 {'Ground': 0},
 {'Start Date': 0},
 {'Match_ID': 0},
 {'Country': 0}]

- RPO is a non-descriptive heading for "Runs Per Over", I will change this to run_rate
- Inns is a non-descriptive name for the innings in which a team has batted. There are some 0 values which are potential null values. For now we'll leave as they are and compare with the bat_1st column in ODI_match_clean.
- Some columns headings have white spaces.
- Target has a lot of null values. All other columns have no null values.
- Score and RPO columns are `object` type which we will convert to float.
- Score is sometimes given in the form "120/8", meaning 8 wickets were taken. We'll separate this out to have score and wickets columns in numerical type.
- 554 matches are represented by 2 rows of data, and 188 matches are represented by a single row of data. It would be tidier if we represented each game in a single row of data.
- Start date is not in datetime format
- Match_ID has a standard format starting with 'ODI #', we can remove this prefix.
- Opposition country in format "v Country", we can remove the "v " prefix
- Unnamed:\_0 likely was an index value and can be dropped.
- RPO and Result columns have 20 and 3 "-" values in the dataset which we will replace with NaN.

## Assessment Summary
***
### `df_match` Cleanliness
- Column names include capital letters and whitsepace characters.
- `Unnamed: 0` appears to be an old index which is no longer useful.
- `BR` non-descriptive name for "Balls Remaining"
- `BR` has 606 null value entries
- "-" is used in place of NaN in `Margin`, `Toss`, `Bat` and `Result` columns
- `Start Date` not in datetime format.
- `Match_ID` has common prefix "ODI #".
- `Margin` has "runs" and "wickets" suffixes to describe the winning method. Need to separate these to convert to integer.
- `Opposition` column has common prefix "v _Country_"
- `Result` has multiple entries likely to mean match abandoned. \['n/r', 'aban', 'canc'\].
- 563 matches are represented by 2 rows of data, 196 matches are represented by a single row of data. Most data can be inferred from the single row but some might be missing.

### `df_totals` Cleanliness
- Column names include capital letters and whitsepace characters.
- `Unnamed: 0` appears to be an old index which is no longer useful.
- `Score` is type object because it is sometimes given in the format "###/#" where the number after "/" is the amount of wickets lost. 
- `Result` and `RPO` have "-" in place of NaN values
- `Inns` is a non-descriptive term for "innings batted".
- `Inns` has some zero values associated with abandoned matches
- `Start Date` not in datetime format.
- `Match_ID` has common prefix "ODI #".
- 554 matches are represented by 2 rows of data, and 188 matches are represented by a single row of data. Most data can be inferred from the single row but some might be missing.

### Tidiness
- Columns duplicated between `df_match` and `df_totals`: `Start Date`, `Ground`, `Match_ID`, `Country`, `Country_ID`, `Opposition`, `Result`.
- Some data across both tables can be inferred from other columns and therefore are not independant variables. Related columns: `Score`, `Overs`, `RPO`, `BR`, `Target`, `Margin`
- Data can be restructured into 2 tables with only the `Match_ID` column common between them. 
***
### Next Steps
 - Restructure data so that we have 2 new DataFrames.
 - `df_match_clean` will have the following columns


## Cleaning & Testing

### Assessment
 - Columns duplicated between `df_match` and `df_totals`: `Start Date`, `Ground`, `Match_ID`, `Country`, `Country_ID`, `Opposition`, `Result`.
 - Some data across both tables can be inferred from other columns and therefore are not independant variables. Related columns: `Score`, `Overs`, `RPO`, `BR`, `Target`, `Margin`
 - Data can be restructured into 2 tables with only the `Match_ID` column common between them. 
### Define
 - Restructure data so that we have 2 new DataFrames; `df_match_clean`, `df_scores_clean`.
 - `df_match_clean`:
     - columns: `match_id`, `start_date`, `team1`, `team2`, `ground`, `result`
     - Each match represented by a single row
     - `result` column will contain the winning team for all completed games that were not tied.
 - `df_scores_clean`:
     - columns: `match_id`, `team`, `opposition`, `toss`, `innings`, `score`, `overs`, `rpo`, `result`, `margin`, `method`
     - Each match will be represented by two rows of data, one for each team.
     - `result` column will contain "win, lose, draw, n/r" categorical data.
 

In [7]:
print(df_totals.columns)
df_match.columns

Index(['Unnamed: 0', 'Score', 'Overs', 'RPO', 'Target', 'Inns', 'Result',
       'Opposition', 'Ground', 'Start Date', 'Match_ID', 'Country',
       'Country_ID'],
      dtype='object')


Index(['Unnamed: 0', 'Result', 'Margin', 'BR', 'Toss', 'Bat', 'Opposition',
       'Ground', 'Start Date', 'Match_ID', 'Country', 'Country_ID'],
      dtype='object')

### Clean & Test

#### Define
- Make all columns lower case with whitespace removed
- Change values of '-' to NaN
- Convert score and RPO from string to float. We're using float here to avoid errors when converting NaN.
- Rename RPO and Inns to run_rate and innings
- Change start date to datetime format
- Remove 'v ' prefix from opposition series
- Remove 'ODI # ' prefix from match_id series
- Drop unnamed:\_0 column
- Combine data from each match into single rows of data.

In [189]:
df_totals.head()

,Unnamed: 0,Score,Overs,RPO,Target,Inns,Result,Opposition,Ground,Start Date,Match_ID,Country,Country_ID
0,412,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,680,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,413,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,681,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,117,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


In [190]:
renameCols(df_totals)
df_totals.head()

,unnamed:_0,score,overs,rpo,target,inns,result,opposition,ground,start_date,match_id,country,country_id
0,412,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,680,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,413,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,681,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,117,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


In [191]:
col_list = []
[col_list.append(k) for i in countVal(df_totals,'-') for k in i]
replaceNaN(df_totals, col_list, '-')

for col in col_list:
    assert (df_totals[col] != '-').all()
    
countVal(df_totals,'-')

[{'score': 0},
 {'rpo': 0.0},
 {'result': 0.0},
 {'opposition': 0},
 {'ground': 0},
 {'start_date': 0},
 {'match_id': 0},
 {'country': 0}]

In [192]:
df_totals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 13 columns):
unnamed:_0    1296 non-null int64
score         1296 non-null object
overs         1296 non-null float64
rpo           1276 non-null object
target        620 non-null float64
inns          1296 non-null int64
result        1293 non-null object
opposition    1296 non-null object
ground        1296 non-null object
start_date    1296 non-null object
match_id      1296 non-null object
country       1296 non-null object
country_id    1296 non-null int64
dtypes: float64(2), int64(3), object(8)
memory usage: 131.7+ KB


`countVal` retuns 0 for all object type columns, there is no assertion error raised when testing for '-' and we have introduced the expected amount of NaN values into the rpo and result columns. Therefore we can be confident that our `replaceNaN()` function has performed correctly.

In [193]:
# df_totals.query('score.str.contains("/")', engine='python')

In [194]:
df_totals['wickets'] = np.where(df_totals.score.str.contains('/'), df_totals.score, np.NaN)
df_totals.head()

,unnamed:_0,score,overs,rpo,target,inns,result,opposition,ground,start_date,match_id,country,country_id,wickets
0,412,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7,NaN
1,680,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6,NaN
2,413,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7,NaN
3,681,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6,NaN
4,117,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8,NaN


In [195]:
for i in range(df_totals.shape[0]):
    try:
        df_totals['wickets'][i] = df_totals.score.str.split('/')[i][1]
    except:
        df_totals['wickets'][i] = np.NaN
df_totals.head(10)

/home/ding/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/ding/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,unnamed:_0,score,overs,rpo,target,inns,result,opposition,ground,start_date,match_id,country,country_id,wickets
0,412,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7,NaN
1,680,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6,NaN
2,413,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7,NaN
3,681,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6,NaN
4,117,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8,NaN
5,1076,305/5,50.0,6.10,NaN,1,won,v Sri Lanka,Melbourne,11 Jan 2013,ODI # 3317,Australia,2,5
6,682,316/9,50.0,6.32,326.0,2,lost,v England,Rajkot,11 Jan 2013,ODI # 3318,India,6,9
7,836,325/4,50.0,6.50,NaN,1,won,v India,Rajkot,11 Jan 2013,ODI # 3318,England,1,4
8,118,172/2,40.1,4.28,171.0,2,won,v Australia,Adelaide,13 Jan 2013,ODI # 3319,SriLanka,8,2
9,1077,170,46.5,3.62,NaN,1,lost,v Sri Lanka,Adelaide,13 Jan 2013,ODI # 3319,Australia,2,NaN


In [196]:
df_totals.dtypes

unnamed:_0      int64
score          object
overs         float64
rpo            object
target        float64
inns            int64
result         object
opposition     object
ground         object
start_date     object
match_id       object
country        object
country_id      int64
wickets        object
dtype: object

In [197]:
df_totals.score = df_totals.score.str.extract('(\d+)').astype(float)
df_totals.head(10)

,unnamed:_0,score,overs,rpo,target,inns,result,opposition,ground,start_date,match_id,country,country_id,wickets
0,412,250.0,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7,NaN
1,680,165.0,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6,NaN
2,413,157.0,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7,NaN
3,681,167.0,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6,NaN
4,117,198.0,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8,NaN
5,1076,305.0,50.0,6.10,NaN,1,won,v Sri Lanka,Melbourne,11 Jan 2013,ODI # 3317,Australia,2,5
6,682,316.0,50.0,6.32,326.0,2,lost,v England,Rajkot,11 Jan 2013,ODI # 3318,India,6,9
7,836,325.0,50.0,6.50,NaN,1,won,v India,Rajkot,11 Jan 2013,ODI # 3318,England,1,4
8,118,172.0,40.1,4.28,171.0,2,won,v Australia,Adelaide,13 Jan 2013,ODI # 3319,SriLanka,8,2
9,1077,170.0,46.5,3.62,NaN,1,lost,v Sri Lanka,Adelaide,13 Jan 2013,ODI # 3319,Australia,2,NaN


In [198]:
def toFloat(df, col_list):
    '''
    Change data types in col_list to type float.
    '''
    for col in col_list:
        df[col] = df[col].astype(float)
    
toFloat(df_totals, ['rpo', 'wickets'])

In [199]:
def renameCols2(df, old_name_list, new_name_list):
    '''
    Rename columns in old_name_list with names in new_name_list.
    Pre-condition: len(old_name_list) == len(new_name_list)
    '''
    for i in range(len(old_name_list)):
        df.rename(columns={old_name_list[i]:new_name_list[i]}, inplace=True)

renameCols2(df_totals, ['rpo', 'inns', 'country', 'opposition'], ['run_rate', 'innings', 'country1', 'country2'])
df_totals.head()

,unnamed:_0,score,overs,run_rate,target,innings,result,country2,ground,start_date,match_id,country1,country_id,wickets
0,412,250.0,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7,NaN
1,680,165.0,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6,NaN
2,413,157.0,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7,NaN
3,681,167.0,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6,NaN
4,117,198.0,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8,NaN


In [200]:
matchPrefix(df_totals, 'match_id')
oppPrefix(df_totals, 'country2')
df_totals.head()

,unnamed:_0,score,overs,run_rate,target,innings,result,country2,ground,start_date,match_id,country1,country_id,wickets
0,412,250.0,48.3,5.15,NaN,1,won,India,Kolkata,3 Jan 2013,3315,Pakistan,7,NaN
1,680,165.0,48.0,3.43,251.0,2,lost,Pakistan,Kolkata,3 Jan 2013,3315,India,6,NaN
2,413,157.0,48.5,3.21,168.0,2,lost,India,Delhi,6 Jan 2013,3316,Pakistan,7,NaN
3,681,167.0,43.4,3.82,NaN,1,won,Pakistan,Delhi,6 Jan 2013,3316,India,6,NaN
4,117,198.0,40.0,4.95,306.0,2,lost,Australia,Melbourne,11 Jan 2013,3317,SriLanka,8,NaN


In [201]:
df_totals.start_date = pd.to_datetime(df_totals.start_date)
df_totals.drop('unnamed:_0', axis=1, inplace=True)
df_totals['team_won'] = np.where(df_totals.result == 'won', df_totals.country1, 
                                np.where(df_totals.result == 'lost', df_totals.country2, 
                                        np.where(df_totals.result == 'n/r', 'n/r', 
                                                np.where(df_totals.result == 'tied', 'tied', None))))
df_totals.head()

,score,overs,run_rate,target,innings,result,country2,ground,start_date,match_id,country1,country_id,wickets,team_won
0,250.0,48.3,5.15,NaN,1,won,India,Kolkata,2013-01-03,3315,Pakistan,7,NaN,Pakistan
1,165.0,48.0,3.43,251.0,2,lost,Pakistan,Kolkata,2013-01-03,3315,India,6,NaN,Pakistan
2,157.0,48.5,3.21,168.0,2,lost,India,Delhi,2013-01-06,3316,Pakistan,7,NaN,India
3,167.0,43.4,3.82,NaN,1,won,Pakistan,Delhi,2013-01-06,3316,India,6,NaN,India
4,198.0,40.0,4.95,306.0,2,lost,Australia,Melbourne,2013-01-11,3317,SriLanka,8,NaN,Australia


In [202]:
df_totals.query('team_won == None')

,score,overs,run_rate,target,innings,result,country2,ground,start_date,match_id,country1,country_id,wickets,team_won


In [203]:
col_order = ['match_id', 'country1', 'country2', 'team_won', 'score', 'wickets', 'target', 'overs', 'run_rate', 'innings',
       'ground', 'start_date', 'country_id', 'result']
len(df_totals.columns) == len(col_order)
df_totals = df_totals[col_order]

In [204]:
df_totals.shape
df_totals.match_id.duplicated().sum()

554

In [205]:
df_totals['country2_score'] = df_totals.target - 1
df_totals.head(10)
# df_totals.info()

,match_id,country1,country2,team_won,score,wickets,target,overs,run_rate,innings,ground,start_date,country_id,result,country2_score
0,3315,Pakistan,India,Pakistan,250.0,NaN,NaN,48.3,5.15,1,Kolkata,2013-01-03,7,won,NaN
1,3315,India,Pakistan,Pakistan,165.0,NaN,251.0,48.0,3.43,2,Kolkata,2013-01-03,6,lost,250.0
2,3316,Pakistan,India,India,157.0,NaN,168.0,48.5,3.21,2,Delhi,2013-01-06,7,lost,167.0
3,3316,India,Pakistan,India,167.0,NaN,NaN,43.4,3.82,1,Delhi,2013-01-06,6,won,NaN
4,3317,SriLanka,Australia,Australia,198.0,NaN,306.0,40.0,4.95,2,Melbourne,2013-01-11,8,lost,305.0
5,3317,Australia,Sri Lanka,Australia,305.0,5.0,NaN,50.0,6.10,1,Melbourne,2013-01-11,2,won,NaN
6,3318,India,England,England,316.0,9.0,326.0,50.0,6.32,2,Rajkot,2013-01-11,6,lost,325.0
7,3318,England,India,England,325.0,4.0,NaN,50.0,6.50,1,Rajkot,2013-01-11,1,won,NaN
8,3319,SriLanka,Australia,SriLanka,172.0,2.0,171.0,40.1,4.28,2,Adelaide,2013-01-13,8,won,170.0
9,3319,Australia,Sri Lanka,Sri Lanka,170.0,NaN,NaN,46.5,3.62,1,Adelaide,2013-01-13,2,lost,NaN


In [206]:
df_totals[df_totals.match_id.duplicated(keep='last')]
# df_totals[df_totals.match_id.value_counts() == 1]
# df_totals.query('innings == 0')
# df_totals.innings.value_counts()
df_totals.rename(columns={'score':'country1_score'}, inplace=True)

In [207]:
col_order = ['match_id', 'country1', 'country1_score', 'country2', 'country2_score', 'team_won', 'wickets', 'target', 'overs', 'run_rate', 'innings',
       'ground', 'start_date', 'country_id', 'result']
len(df_totals.columns) == len(col_order)
df_totals = df_totals[col_order]
df_totals.head()

,match_id,country1,country1_score,country2,country2_score,team_won,wickets,target,overs,run_rate,innings,ground,start_date,country_id,result
0,3315,Pakistan,250.0,India,NaN,Pakistan,NaN,NaN,48.3,5.15,1,Kolkata,2013-01-03,7,won
1,3315,India,165.0,Pakistan,250.0,Pakistan,NaN,251.0,48.0,3.43,2,Kolkata,2013-01-03,6,lost
2,3316,Pakistan,157.0,India,167.0,India,NaN,168.0,48.5,3.21,2,Delhi,2013-01-06,7,lost
3,3316,India,167.0,Pakistan,NaN,India,NaN,NaN,43.4,3.82,1,Delhi,2013-01-06,6,won
4,3317,SriLanka,198.0,Australia,305.0,Australia,NaN,306.0,40.0,4.95,2,Melbourne,2013-01-11,8,lost


In [208]:
dups = dupIndex(df_totals, 'match_id')

In [209]:
df_bat1st = df_totals.query('country2_score != country2_score')
df_bat1st.head()
# dupIndex(df_totals, 'match_id')


,match_id,country1,country1_score,country2,country2_score,team_won,wickets,target,overs,run_rate,innings,ground,start_date,country_id,result
0,3315,Pakistan,250.0,India,NaN,Pakistan,NaN,NaN,48.3,5.15,1,Kolkata,2013-01-03,7,won
3,3316,India,167.0,Pakistan,NaN,India,NaN,NaN,43.4,3.82,1,Delhi,2013-01-06,6,won
5,3317,Australia,305.0,Sri Lanka,NaN,Australia,5.0,NaN,50.0,6.10,1,Melbourne,2013-01-11,2,won
7,3318,England,325.0,India,NaN,England,4.0,NaN,50.0,6.50,1,Rajkot,2013-01-11,1,won
9,3319,Australia,170.0,Sri Lanka,NaN,Sri Lanka,NaN,NaN,46.5,3.62,1,Adelaide,2013-01-13,2,lost


In [210]:
df_totals.drop(dupIndex(df_totals, 'match_id'), inplace=True, axis=0)

In [211]:
df_bat1st.shape[0], df_totals.shape[0]

(676, 742)

In [212]:
df_bat1st.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 676 entries, 0 to 1295
Data columns (total 15 columns):
match_id          676 non-null int64
country1          676 non-null object
country1_score    656 non-null float64
country2          676 non-null object
country2_score    0 non-null float64
team_won          675 non-null object
wickets           474 non-null float64
target            0 non-null float64
overs             676 non-null float64
run_rate          656 non-null float64
innings           676 non-null int64
ground            676 non-null object
start_date        676 non-null datetime64[ns]
country_id        676 non-null int64
result            675 non-null object
dtypes: datetime64[ns](1), float64(6), int64(3), object(5)
memory usage: 84.5+ KB


In [213]:
df_totals.to_csv('../Data/interim/ODI_totals_clean.csv', index=False)
df_bat1st.to_csv('../Data/interim/ODI_bat1st.csv', index=False)

In [309]:
df_match = pd.read_csv('../Data/interim/ODI_match_clean.csv')
df_totals = pd.read_csv('../Data/interim/ODI_totals_clean.csv')

In [310]:
df_match.shape[0], df_totals.shape[0]

(743, 742)

Now that we have cleaned both df_totals and df_match and made the match_id column in each dataframe a unique key, we can merge the 2 data frames so that all the match_data is contained within a single table. df_totals and df_match have 742 and 743 rows of data respectively so we know there is at least one match_id that is unique to df_match. It is quite possible that there are more unique match_id values in each df so we will perform an outer merge to begin with and assess the size.

Before carrying out the merge I will label all the columns in each dataframe with "\_m" for match and "\_t" for totals so that we can keep track of the source of the data. This will enable us to check the success of the merge by matching data in euqivalent columns.

In [311]:
# def renameCols3(df, suffix):
#     '''
#     Add suffix to every column name in df.
#     '''
#     df.rename(lambda c: c + suffix, axis=1, inplace=True)

# renameCols3(df_match, '_m')
# renameCols3(df_totals, '_t')
# df_match.head()
# df_totals.head()

In [312]:
# df_merge = df_match.merge(df_totals, how='outer', left_on='match_id_m', right_on='match_id_t', indicator=True)
df_merge = df_match.merge(df_totals, how='inner', on='match_id', indicator=True)

In [313]:
df_merge.head()

,match_id,country1_x,country2_x,team_won_x,toss,bat,win_margin,win_method,balls_remaining,ground_x,...,wickets,target,overs,run_rate,innings,ground_y,start_date_y,country_id_y,result_y,_merge
0,3315,Pakistan,India,Pakistan,lost,1st,85.0,runs,NaN,Kolkata,...,NaN,NaN,48.3,5.15,1,Kolkata,2013-01-03,7,won,both
1,3316,Pakistan,India,India,lost,2nd,10.0,runs,NaN,Delhi,...,NaN,168.0,48.5,3.21,2,Delhi,2013-01-06,7,lost,both
2,3317,SriLanka,Australia,Australia,lost,2nd,107.0,runs,NaN,Melbourne,...,NaN,306.0,40.0,4.95,2,Melbourne,2013-01-11,8,lost,both
3,3318,India,England,England,lost,2nd,9.0,runs,NaN,Rajkot,...,9.0,326.0,50.0,6.32,2,Rajkot,2013-01-11,6,lost,both
4,3319,SriLanka,Australia,SriLanka,won,2nd,8.0,wickets,59.0,Adelaide,...,2.0,171.0,40.1,4.28,2,Adelaide,2013-01-13,8,won,both


In [314]:
df_merge._merge.value_counts()

both          742
right_only      0
left_only       0
Name: _merge, dtype: int64

In [315]:
df_merge.shape

(742, 28)

`df_merge` has 743 rows and the `value_counts()` method tells us that only 1 row of data comes from `df_match` only. 

In [316]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 741
Data columns (total 28 columns):
match_id           742 non-null int64
country1_x         742 non-null object
country2_x         742 non-null object
team_won_x         742 non-null object
toss               742 non-null object
bat                740 non-null object
win_margin         702 non-null float64
win_method         702 non-null object
balls_remaining    348 non-null float64
ground_x           742 non-null object
start_date_x       742 non-null object
country_id_x       742 non-null int64
result_x           740 non-null object
country1_y         742 non-null object
country1_score     733 non-null float64
country2_y         742 non-null object
country2_score     359 non-null float64
team_won_y         740 non-null object
wickets            491 non-null float64
target             359 non-null float64
overs              742 non-null float64
run_rate           733 non-null float64
innings            742 non-null

Now we'll test the merge to make sure all equivalent colums are matching. The list of columns that should match are:
- result_x : result_y
\['country1_x', 'country2_x', 'team_won_x', 'toss', 'bat', 'win_margin', 'win_method', 'balls_remaining', 'ground_x', 'start_date_x', 'country_id_x', 'result_x', 'country1_y', 'country1_score', 'country2_y', 'country2_score', 'team_won_y', 'wickets', 'target', 'overs', 'run_rate', 'innings', 'ground_y', 'start_date_y', 'country_id_y', 'result_y', '_merge']

In [317]:
df_merge[df_merge.team_won_x != df_merge.team_won_y][['country1_x', 'country2_x', 'team_won_x', 'ground_x', 'start_date_x', 'country_id_x', 'result_x', 'country1_y', 'country1_score', 'country2_y', 'country2_score', 'team_won_y', 'overs', 'ground_y', 'start_date_y', 'country_id_y', 'result_y']].head(15)

,country1_x,country2_x,team_won_x,ground_x,start_date_x,country_id_x,result_x,country1_y,country1_score,country2_y,country2_score,team_won_y,overs,ground_y,start_date_y,country_id_y,result_y
740,Afghanistan,Ireland,Ireland,Belfast,2019-05-19,40,NaN,Afghanistan,120.0,Ireland,210.0,NaN,33.0,Belfast,2019-05-19,40,NaN
741,Pakistan,England,England,Leeds,2019-05-19,7,NaN,Pakistan,193.0,England,351.0,NaN,31.5,Leeds,2019-05-19,7,NaN


In [318]:
col_list1 = ['country1_x', 'country2_x', 'team_won_x', 'ground_x', 'start_date_x', 'country_id_x', 'result_x']
col_list2 = ['country1_y', 'country2_y', 'team_won_y', 'ground_y', 'start_date_y', 'country_id_y', 'result_y']

def checkMerge(df, col_list1, col_list2):
    '''
    Checks equality between columns in col_list1 and col_list2 in df.
    Precondition: len(col_list1) == len(col_list2)
    '''
    for i in range(len(col_list1)):
        try:
            assert((df[col_list1[i]] != df[col_list2[i]]).sum() == 0)
        except:
            print("AssertionError:", col_list1[i], "!=", col_list2[i])
            
checkMerge(df_merge,col_list1, col_list2)

AssertionError: team_won_x != team_won_y
AssertionError: result_x != result_y


In [319]:
df_merge.query('team_won_x != team_won_y or result_x != result_y')[['match_id','country1_x', 'country2_x', 'team_won_x', 'ground_x', 'start_date_x', 'country_id_x', 'result_x', 'country1_y', 'country2_y', 'team_won_y', 'ground_y', 'start_date_y', 'country_id_y', 'result_y']]

,match_id,country1_x,country2_x,team_won_x,ground_x,start_date_x,country_id_x,result_x,country1_y,country2_y,team_won_y,ground_y,start_date_y,country_id_y,result_y
740,4139,Afghanistan,Ireland,Ireland,Belfast,2019-05-19,40,NaN,Afghanistan,Ireland,NaN,Belfast,2019-05-19,40,NaN
741,4140,Pakistan,England,England,Leeds,2019-05-19,7,NaN,Pakistan,England,NaN,Leeds,2019-05-19,7,NaN


In [320]:
df_totals.query('match_id == 4139 or match_id == 4140')
df_merge.query('match_id == 4139 or match_id == 4140')

,match_id,country1_x,country2_x,team_won_x,toss,bat,win_margin,win_method,balls_remaining,ground_x,...,wickets,target,overs,run_rate,innings,ground_y,start_date_y,country_id_y,result_y,_merge
740,4139,Afghanistan,Ireland,Ireland,won,2nd,NaN,NaN,NaN,Belfast,...,8.0,211.0,33.0,3.63,2,Belfast,2019-05-19,40,NaN,both
741,4140,Pakistan,England,England,lost,2nd,NaN,NaN,NaN,Leeds,...,6.0,352.0,31.5,6.06,2,Leeds,2019-05-19,7,NaN,both


In [321]:
df_merge.loc[[740,741],["result_x", "result_y"]] = "lost"
df_merge.loc[[740,741],["team_won_x", "team_won_y"]] = df_totals.loc[[740,741],"country2"]
# df_totals[(df_totals.match_id == 4139)].result = "lost"

In [322]:
df_totals.query('match_id == 4139 or match_id == 4140')

,match_id,country1,country1_score,country2,country2_score,team_won,wickets,target,overs,run_rate,innings,ground,start_date,country_id,result
740,4139,Afghanistan,120.0,Ireland,210.0,NaN,8.0,211.0,33.0,3.63,2,Belfast,2019-05-19,40,NaN
741,4140,Pakistan,193.0,England,351.0,NaN,6.0,352.0,31.5,6.06,2,Leeds,2019-05-19,7,NaN


In [323]:
# Check series for equality again
checkMerge(df_merge,col_list1, col_list2)

In [324]:
df_merge.query('team_won_x != team_won_y or result_x != result_y')[['result_y', 'result_x']]

,result_y,result_x


In [325]:
match = set(df_match.match_id.values)
totals = set(df_totals.match_id.values)
[x for x in match if x not in totals]

[3729]

In [326]:
df_match.query('match_id == 3729')

,match_id,country1,country2,team_won,toss,bat,win_margin,win_method,balls_remaining,ground,start_date,country_id,result
383,3729,Pakistan,New Zealand,n/r,NaN,NaN,NaN,NaN,NaN,Napier,2016-01-28,7,n/r


The match which is not present in both df_match and df_total has many NaN values and result = "n/r". It will therefore not be useful for analysis so I will not investigate to retrieve the data.

In [327]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 741
Data columns (total 28 columns):
match_id           742 non-null int64
country1_x         742 non-null object
country2_x         742 non-null object
team_won_x         742 non-null object
toss               742 non-null object
bat                740 non-null object
win_margin         702 non-null float64
win_method         702 non-null object
balls_remaining    348 non-null float64
ground_x           742 non-null object
start_date_x       742 non-null object
country_id_x       742 non-null int64
result_x           742 non-null object
country1_y         742 non-null object
country1_score     733 non-null float64
country2_y         742 non-null object
country2_score     359 non-null float64
team_won_y         742 non-null object
wickets            491 non-null float64
target             359 non-null float64
overs              742 non-null float64
run_rate           733 non-null float64
innings            742 non-null

Now that our data is merged successfully and we know all "\_x" and "\_y" columns match exactly, we can drop and rename the duplicated columns.

In [328]:
df_merge.drop(columns=col_list2, inplace=True, axis=1)
col_list_new = ['country1', 'country2', 'team_won', 'ground', 'start_date', 'country_id', 'result']
renameCols2(df_merge, col_list1, col_list_new)
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 741
Data columns (total 21 columns):
match_id           742 non-null int64
country1           742 non-null object
country2           742 non-null object
team_won           742 non-null object
toss               742 non-null object
bat                740 non-null object
win_margin         702 non-null float64
win_method         702 non-null object
balls_remaining    348 non-null float64
ground             742 non-null object
start_date         742 non-null object
country_id         742 non-null int64
result             742 non-null object
country1_score     733 non-null float64
country2_score     359 non-null float64
wickets            491 non-null float64
target             359 non-null float64
overs              742 non-null float64
run_rate           733 non-null float64
innings            742 non-null int64
_merge             742 non-null category
dtypes: category(1), float64(8), int64(3), object(9)
memory usage: 12

In [ ]:
col_order = ['match_id', 'country1', 'country1_score', 'country2', 'country2_score', 'team_won', 'wickets', 'target', 'overs', 'run_rate', 'innings',
       'ground', 'start_date', 'country_id', 'result']

In [53]:
# df_match #[(df_match.BR.isnull()==False)].head() ODI # 3324
# df_totals.query('Match_ID == "ODI # 3324"')
# df_match.Result.value_counts()`
# df_match.Margin.value_counts()

In [107]:
# import re
# # df_match['win_wick'] = (np.where(df_match.Margin.str.endswith("wickets"), True, False))
# # df_match['win_wick'] = np.where("wick" in df_match.Margin, int(re.findall("\d+", df_match.Margin)), 'NaN')
# # re.findall("\d+", str(df_match.Margin))
# # df_match.Margin.str.extract('(\d+)').astype(int)
# # try:
# #     df_match.Margin.str.extract('(\d+)').astype(int)
# # except:
# #     df_match.Margin.str.extract('(\d+)')
# # df_match[(df_match.win_wick == True)].index
# # for i in range(df_match.shape[0]):
# #     try:
# # df_match.Margin = np.where(df_match.Margin == '-', np.NaN, df_match.Margin)

# # df_match['win_margin'] = df_match.Margin.apply(lambda x: x.split(' ')[0])
# # df_match['win_margin'] = df_match.Margin.apply(lambda x: x.extract('(\d+)'))
# df_match['win_margin'] = df_match.margin.str.extract('(\d+)').astype(float)
# df_match['win_method'] = df_match.margin.str.extract('([a-zA-Z]+)')
# #         df_match['win_method'] = df_match.Margin.apply(lambda x: x.split(' ')[1])
# #     except:
# #         df_match['win_margin'][i] = None
# #         df_match['win_method'][i] = None
# df_match.query('win_method == "runs"')
# # df_match.info()

- RPO is a non-descriptive heading for runs per over, Inns non-descriptive for 'innings', some columns headings have white spaces.
- Target has a lot of null values which should be handled. All other columns have no null values.
- Score is given in string format and contains information about wickets. We'll need to separate this data out and convert to integer.
- 554 matches are represented by 2 rows of data, and 188 matches are represented by a single row of data. It would be tidier if we represented each game in a single row of data.
- Start date is not in datetime format
- Match_ID has a standard format starting with 'ODI #', we can remove this prefix.
- Opposition country in format "v Country", we can remove the "v " prefix
- Unnamed: 0 likely was an index value and can be dropped.

We can see that both dataframes share a lot of information about each match, it would be useful to merge these together into a single dataframe. Since each most Match_ID's have entries for both teams, they are duplicated exactly twice and therefore cannot be used as is a primary or foreign key for the merge. Therefore, we need to merge on multiple columns. I will used the Match_ID and Country_ID columns for the merge, and check for validity using the match Result columns.

Before carrying out the merge I will rename all columns in both dataframes so that they are all lower case and have no spaces, and mark each column with \_m for match and \_t for totals so that we know what the source df is.

> - Here we see the win magain is provided as a string and is given as 'runs' or 'wickets. We'll want to separate these out and convert to integer type so that we can perform more meaningful analysis.
> - Since the outcomes for 'Toss' and 'Bat' are binary we can convert these columns to represent boolean values.
> - 'Start Date' is type string so it would be useful to convert this to date_time.
> - 'Match_ID' has a consistent format "ODI #" as a preface to the match id number which we can drop to make life easier for ourselves later on.
> - The unnamed column seems to have previously been an index so can be removed.

In [294]:
df_totals.columns

Index(['Unnamed: 0', 'Score', 'Overs', 'RPO', 'Target', 'Inns', 'Result',
       'Opposition', 'Ground', 'Start Date', 'Match_ID', 'Country',
       'Country_ID'],
      dtype='object')

In [295]:
# for c in df_totals.columns:
df_totals.rename(lambda c: c.lower().replace(' ', '_') + '_t', axis=1, inplace=True)
df_totals.head()

,unnamed:_0_t,score_t,overs_t,rpo_t,target_t,inns_t,result_t,opposition_t,ground_t,start_date_t,match_id_t,country_t,country_id_t
0,412,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,680,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,413,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,681,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,117,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


In [296]:
# for c in df_totals.columns:
df_match.rename(lambda c: c.lower().replace(' ', '_') + '_m', axis=1, inplace=True)
df_match.head()

,unnamed:_0_m,result_m,margin_m,br_m,toss_m,bat_m,opposition_m,ground_m,start_date_m,match_id_m,country_m,country_id_m
0,418,won,85 runs,NaN,lost,1st,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,692,lost,85 runs,NaN,won,2nd,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,419,lost,10 runs,NaN,lost,2nd,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,693,won,10 runs,NaN,won,1st,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,121,lost,107 runs,NaN,lost,2nd,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


> The number of rows in each df are not equal so we will use and inner join so that only matches present in both df's are merged into the new_df.
>
> To view the success of this merge I will check the result, country and opposition collumns for equality. We will hopefully see no mismatched data here.

In [297]:
df_new = df_match.merge(df_totals, left_on=['match_id_m', 'country_id_m'], right_on=['match_id_t', 'country_id_t'], how='inner')
df_new.head()

,unnamed:_0_m,result_m,margin_m,br_m,toss_m,bat_m,opposition_m,ground_m,start_date_m,match_id_m,...,rpo_t,target_t,inns_t,result_t,opposition_t,ground_t,start_date_t,match_id_t,country_t,country_id_t
0,418,won,85 runs,NaN,lost,1st,v India,Kolkata,3 Jan 2013,ODI # 3315,...,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,692,lost,85 runs,NaN,won,2nd,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,...,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,419,lost,10 runs,NaN,lost,2nd,v India,Delhi,6 Jan 2013,ODI # 3316,...,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,693,won,10 runs,NaN,won,1st,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,...,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,121,lost,107 runs,NaN,lost,2nd,v Australia,Melbourne,11 Jan 2013,ODI # 3317,...,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


In [298]:
df_new.result_m.eq(df_new.result_t).sum() == df_new.shape[0]

True

In [299]:
df_new['merge_check'] = np.where((df_new.result_m == df_new.result_m) & \
                                 (df_new.country_m == df_new.country_t) & \
                                 (df_new.opposition_m == df_new.opposition_t), True, False)
df_new.merge_check.sum() == df_new.shape[0]

True

> Here, we have created a new column which contains True if, and only if, all three logical statements are true. All values in this column are True so we can be confident that we have merged sucessfully.
>
> Now we can drop columns that are either duplicates of each other following the merge or not useful for analysis.

In [300]:
drop_cols = ['unnamed:_0_m', 'result_m', 'opposition_m', 'ground_m', 'start_date_m', 'match_id_m', \
             'country_m', 'country_id_m', 'unnamed:_0_t', 'merge_check']
df_new.drop(columns=drop_cols, inplace=True)
df_new.head()

,margin_m,br_m,toss_m,bat_m,score_t,overs_t,rpo_t,target_t,inns_t,result_t,opposition_t,ground_t,start_date_t,match_id_t,country_t,country_id_t
0,85 runs,NaN,lost,1st,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,85 runs,NaN,won,2nd,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,10 runs,NaN,lost,2nd,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,10 runs,NaN,won,1st,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,107 runs,NaN,lost,2nd,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


In [301]:
df_new.rename(lambda c: c[:-2], axis=1, inplace=True)

In [302]:
df_new.head()

,margin,br,toss,bat,score,overs,rpo,target,inns,result,opposition,ground,start_date,match_id,country,country_id
0,85 runs,NaN,lost,1st,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,85 runs,NaN,won,2nd,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,10 runs,NaN,lost,2nd,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,10 runs,NaN,won,1st,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,107 runs,NaN,lost,2nd,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


In [303]:
df_new.to_csv('../Data/interim/match_totals_comb.csv')

In [142]:
df_new.shape, df_match.shape, df_totals.shape

((1322, 25), (1322, 12), (1296, 13))

In [143]:
df_new2.shape, df_match.shape, df_totals.shape

((1322, 22), (1322, 12), (1296, 13))

In [222]:
df_new2['Opposition_x'].eq(df_new2['Opposition_y'])==False

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
1292    False
1293    False
1294    False
1295    False
1296    False
1297    False
1298    False
1299    False
1300    False
1301     True
1302    False
1303    False
1304     True
1305    False
1306    False
1307    False
1308    False
1309    False
1310    False
1311    False
1312    False
1313    False
1314    False
1315    False
1316    False
1317    False
1318     True
1319    False
1320    False
1321    False
Length: 1322, dtype: bool

In [25]:
df_bowl.head()

,Unnamed: 0,Overs,Mdns,Runs,Wkts,Econ,Ave,SR,Opposition,Ground,Start Date,Match_ID,Bowler,Player_ID
0,1,8.0,0,57,0,7.12,-,-,v India,Nagpur,18 Dec 2009,ODI # 2933,Suranga Lakmal,49619
1,2,10.0,0,55,2,5.50,27.50,30.0,v India,Kolkata,24 Dec 2009,ODI # 2935,Suranga Lakmal,49619
2,3,-,-,-,-,-,-,-,v India,Delhi,27 Dec 2009,ODI # 2936,Suranga Lakmal,49619
3,4,9.0,1,63,2,7.00,31.50,27.0,v Bangladesh,Dhaka,4 Jan 2010,ODI # 2937,Suranga Lakmal,49619
4,5,8.0,1,48,0,6.00,-,-,v India,Dhaka,5 Jan 2010,ODI # 2938,Suranga Lakmal,49619


In [38]:
df_bowl.query('Bowler == "Oshane Thomas"')

,Unnamed: 0,Overs,Mdns,Runs,Wkts,Econ,Ave,SR,Opposition,Ground,Start Date,Match_ID,Bowler,Player_ID
81,82,9.0,0,83,1,9.22,83.00,54.0,v India,Guwahati,21 Oct 2018,ODI # 4056,Oshane Thomas,914567
82,83,4.0,0,33,1,8.25,33.00,24.0,v India,Thiruvananthapuram,1 Nov 2018,ODI # 4064,Oshane Thomas,914567
83,84,5.0,0,34,1,6.80,34.00,30.0,v Bangladesh,Dhaka,9 Dec 2018,ODI # 4071,Oshane Thomas,914567
84,85,10.0,0,54,3,5.40,18.00,20.0,v Bangladesh,Dhaka,11 Dec 2018,ODI # 4072,Oshane Thomas,914567
85,86,9.0,0,72,1,8.00,72.00,54.0,v England,Bridgetown,20 Feb 2019,ODI # 4096,Oshane Thomas,914567
86,87,5.0,0,32,1,6.40,32.00,30.0,v England,Bridgetown,22 Feb 2019,ODI # 4097,Oshane Thomas,914567
87,88,-,-,-,-,-,-,-,v England,St George's,25 Feb 2019,ODI # 4098,Oshane Thomas,914567
88,89,10.0,0,84,2,8.40,42.00,30.0,v England,St George's,27 Feb 2019,ODI # 4099,Oshane Thomas,914567
89,90,5.1,0,21,5,4.06,4.20,6.2,v England,Gros Islet,2 Mar 2019,ODI # 4103,Oshane Thomas,914567


In [26]:
df_bat.head()

,Unnamed: 0,Bat1,Runs,BF,SR,4s,6s,Opposition,Ground,Start Date,Match_ID,Batsman,Player_ID
0,1,DNB,-,-,-,-,-,v India,Nagpur,18 Dec 2009,ODI # 2933,Oshane Thomas,49619
1,2,DNB,-,-,-,-,-,v India,Kolkata,24 Dec 2009,ODI # 2935,Oshane Thomas,49619
2,3,DNB,-,-,-,-,-,v India,Delhi,27 Dec 2009,ODI # 2936,Oshane Thomas,49619
3,4,DNB,-,-,-,-,-,v Bangladesh,Dhaka,4 Jan 2010,ODI # 2937,Oshane Thomas,49619
4,5,DNB,-,-,-,-,-,v India,Dhaka,5 Jan 2010,ODI # 2938,Oshane Thomas,49619


In [34]:
df_bat.dtypes

Unnamed: 0     int64
Bat1          object
Runs          object
BF            object
SR            object
4s            object
6s            object
Opposition    object
Ground        object
Start Date    object
Match_ID      object
Batsman       object
Player_ID      int64
dtype: object

In [39]:
df_bat.query('Player_ID == 49619').head()

,Unnamed: 0,Bat1,Runs,BF,SR,4s,6s,Opposition,Ground,Start Date,Match_ID,Batsman,Player_ID
0,1,DNB,-,-,-,-,-,v India,Nagpur,18 Dec 2009,ODI # 2933,Oshane Thomas,49619
1,2,DNB,-,-,-,-,-,v India,Kolkata,24 Dec 2009,ODI # 2935,Oshane Thomas,49619
2,3,DNB,-,-,-,-,-,v India,Delhi,27 Dec 2009,ODI # 2936,Oshane Thomas,49619
3,4,DNB,-,-,-,-,-,v Bangladesh,Dhaka,4 Jan 2010,ODI # 2937,Oshane Thomas,49619
4,5,DNB,-,-,-,-,-,v India,Dhaka,5 Jan 2010,ODI # 2938,Oshane Thomas,49619


In [68]:
cols = df_bat.columns
for i in cols:
    print('{} is unique: {}'.format(i, df_bat[i].is_unique))

Unnamed: 0 is unique: True
Bat1 is unique: False
Runs is unique: False
BF is unique: False
SR is unique: False
4s is unique: False
6s is unique: False
Opposition is unique: False
Ground is unique: False
Start Date is unique: False
Match_ID is unique: False
Batsman is unique: False
Player_ID is unique: False


In [24]:
df_players.head()

,Player,ID,Country
0,Gulbadin Naib (c),352048,Afghanistan
1,Rashid Khan (vc),793463,Afghanistan
2,Aftab Alam,440963,Afghanistan
3,Asghar Afghan,320652,Afghanistan
4,Dawlat Zadran,516561,Afghanistan


In [27]:
df_ground.head()

,Ground,Span,Mat,Won,Tied,NR,Runs,Wkts,Balls,Ave,RPO
0,"Eden Gardens, Kolkata - India",2013-2017,4,4,0,0,2161,72,2297,30.01,5.64
1,"Feroz Shah Kotla, Delhi - India",2013-2019,4,4,0,0,1789,75,2331,23.85,4.60
2,Melbourne Cricket Ground - Australia,2013-2019,15,15,0,0,7656,217,8482,35.28,5.41
3,"Saurashtra Cricket Association Stadium, Rajkot...",2013-2015,2,2,0,0,1163,26,1200,44.73,5.81
4,Adelaide Oval - Australia,2013-2019,10,10,0,0,4863,157,5645,30.97,5.16


In [31]:
df_ground.NR.value_counts(), df_ground.Tied.value_counts()

(0    81
 1    18
 2     6
 3     1
 Name: NR, dtype: int64, 0    97
 1     8
 2     1
 Name: Tied, dtype: int64)